In [119]:
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
api_key = os.environ['OPENAI_API_KEY']

from langchain_openai import ChatOpenAI
#memory and conversation
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationSummaryMemory
#chain
from langchain.chains import LLMChain
from langchain_core.runnables import RunnableMap
#prompts
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [87]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.9,
    max_tokens=None,
    timeout=None,
)

memory = ConversationSummaryMemory(llm=llm)

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)


prompt = ChatPromptTemplate.from_template(
    "Generate a name to describe a company that makes"
)


In [88]:
#Chain combines LLM with a prompt and carry out a sequence of operations on text and data

#import data
import pandas as pd
df = pd.read_csv('Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [89]:
#create a chain
chain = LLMChain(llm=llm, prompt=prompt)
product = "Queen Size Sheet Set"


# Use `invoke` instead of `run`
response = chain.invoke({"product": product})

# Print the result
print(response)

{'product': 'Queen Size Sheet Set', 'text': "Of course! Could you please provide more details about the company's products or services?"}


In [90]:
from langchain.chains import SimpleSequentialChain

In [91]:


# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [92]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [93]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [94]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Choosing a name for a company that specializes in Queen Size Sheet Sets involves considering factors like brand identity, target audience, and market positioning. Here are a few suggestions:

1. **Queen's Comfort**
2. **Royal Rest Linens**
3. **Majestic Sheets**
4. **Sovereign Sleep**
5. **Elegance Bedding**
6. **Regal Sleepwear**
7. **Queen's Haven**
8. **Dreamy Drapes**
9. **Noble Night Linens**
10. **Crown Comforts**

When choosing a name, it's important to ensure that it's unique, easy to remember, and can be easily associated with the products you are offering. Additionally, checking for domain availability for a matching website can be beneficial.
"Premium supplier of luxurious queen-size sheet sets, blending comfort and elegance for a regal sleeping experience. Perfect for discerning customers."

> Finished chain.


'"Premium supplier of luxurious queen-size sheet sets, blending comfort and elegance for a regal sleeping experience. Perfect for discerning customers."'

In [95]:
##SEQUENTIAL CHAIN

from langchain.chains import SequentialChain


# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )



In [96]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [97]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is dissatisfied with the taste and quality of the product compared to store-bought versions, suspecting it might be from an old batch or counterfeit.',
 'followup_message': "Cher(e) [Nom du client],\n\nMerci d'avoir partagé votre retour d'expérience concernant notre produit. Nous sommes désolés d'apprendre que le goût et la qualité n'ont pas répondu à vos attentes. Sachez que nous prenons très au sérieux les préoccupations de nos clients. Nous nous engageons à n'utiliser que des ingrédients de haute qualité et à respecter des normes de production rigoureuses. Il est possible qu'il s'agisse d'un lot défectueux ou d'u

In [133]:
##ROUTER CHAIN

physics_template = """The answer is physics. Respond back "physics" do not write more.

Here is a question:
{input}"""


math_template = """The answer is math. Respond back "math" do not write more.

Here is a question:
{input}"""

history_template = """The answer is history. Respond back "history" do note write more.

Here is a question:
{input}"""


computerscience_template = """ The answer is "comp sci". Respond back "comp sci" do not write more.

Here is a question:
{input}"""

In [142]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "history", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [143]:
# Create destination chains dynamically
destination_chains = {}
for p_info in prompt_infos:
    prompt_template = ChatPromptTemplate.from_template(template=p_info["prompt_template"])
    chain = LLMChain(llm=llm, prompt=prompt_template)
    destination_chains[p_info["name"]] = chain
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)


#default destination chain
default_prompt = ChatPromptTemplate.from_template("I am not sure how to answer this: {input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)


In [144]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [145]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [156]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
)

# Use LLM to execute the router prompt
router_chain = LLMChain(llm=llm, prompt=router_prompt)


In [157]:
# Example Inputs
example_inputs = [
    {"input": "Tell me about Newton's laws of motion."},
    {"input": "What is 2 + 2?"},
    {"input": "Explain the Industrial Revolution."},
    {"input": "What is a Turing machine?"},
    {"input": "What is the weather like today?"},  # To test the default chain
]

for user_input in example_inputs:
    print(router_chain.predict(input=user_input))
    print("-" * 50)

```json
{
    "destination": "physics",
    "next_inputs": "Tell me about Newton's laws of motion."
}
```
--------------------------------------------------
```json
{
    "destination": "math",
    "next_inputs": "What is 2 + 2?"
}
```
--------------------------------------------------
```json
{
    "destination": "history",
    "next_inputs": "Explain the Industrial Revolution."
}
```
--------------------------------------------------
```json
{
    "destination": "computer science",
    "next_inputs": "What is a Turing machine?"
}
```
--------------------------------------------------
```json
{
    "destination": "DEFAULT",
    "next_inputs": "What is the weather like today?"
}
```
--------------------------------------------------


In [158]:
# Parse the router result and run the appropriate chain
def execute_chain(input_text):
    # Get the destination and modified input from the router
    router_result = router_chain.predict(input=input_text)
    parsed_result = RouterOutputParser().parse(router_result)
    destination = parsed_result["destination"]
    next_inputs = parsed_result["next_inputs"]

    # If destination is in destination_chains, run that chain
    if destination in destination_chains:
        response = destination_chains[destination].predict(input=next_inputs)
    else:  # Fallback to default chain
        response = default_chain.predict(input=next_inputs)
    
    return response

# Example Usage
example_inputs = [
    "Tell me about Newton's laws of motion.",
    "What is 2 + 2?",
    "Explain the Industrial Revolution.",
    "What is a Turing machine?",
    "What is the weather like today?"  # Should fallback to default
]

for input_text in example_inputs:
    response = execute_chain(input_text)
    print(f"Input: {input_text}")
    print(f"Response: {response}")
    print("-" * 50)

Input: Tell me about Newton's laws of motion.
Response: Physics.
--------------------------------------------------
Input: What is 2 + 2?
Response: math
--------------------------------------------------
Input: Explain the Industrial Revolution.
Response: History.
--------------------------------------------------
Input: What is a Turing machine?
Response: comp sci
--------------------------------------------------
Input: What is the weather like today?
Response: I'm sorry, but I can't provide real-time weather updates or information. Please check a reliable weather website or app for the current conditions in your area.
--------------------------------------------------
